# logging
## wechat_challenge setup_logging()
logging类是非常重要的一种打印输出的手法。在深度学习中，需要打印的有acc,loss,time,等等参数
最好按组输出训练参数，并且分组计算acc或者计算别的方法

In [1]:
import logging
import torch

In [ ]:
def setup_logging():
    logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S',
                        level=logging.INFO)
    logger = logging.getLogger(__name__)

    return logger

#build_optimizer
## wechat_challenge build_optimizer

我个人一直有一个疑惑就是optimizer和训练的输出应当如何控制 如何去编写一个训练过程控制器
- build_optimizer 类里卖弄包含了bias和layernorm.weight
- optimizer使用了AdamW初始化 scheduler
- 返回optimizer和scheduler

In [ ]:
def build_optimizer(args, model):
    no_decay = ['bias', 'LayerNorm.weight']
